In [1]:
class SqlValue:
    def __init__(self, value):
        self.value = value
    def to_sql(self):
        return self.value
    
    #binary arithmetic operations
    def __add__(self, rhs):
        if (type(rhs) == int):
            rhs = SqlInt(rhs)
        return SqlAdd(self, rhs)
    
    def __eq__(self, rhs):
        if (type(rhs) == int):
            rhs = SqlInt(rhs)
        if (type(rhs) == bool):
            rhs = SqlBool(rhs)
        return SqlCheckEquality(self, rhs) 
    
    def __gt__(self, rhs):
        if (type(rhs) == int):
            rhs = SqlInt(rhs)
        if (type(rhs) == bool):
            rhs = SqlBool(rhs)
        return SqlCheckGreaterThan(self, rhs)
    
    def __rgt__(self, lhs):
        if (type(lhs) == int):
            lhs = SqlInt(lhs)
        if (type(lhs) == bool):
            lhs = SqlBool(lhs)
        return SqlCheckGreaterThan(lhs, self)

In [2]:
class SqlInt(SqlValue):
    def __init__(self, value):
        super().__init__(value)
        self.sql_type = 'int'

In [3]:
class SqlBool(SqlValue):
    def __init__(self, value):
        super().__init__(value)
        self.sql_type = 'bool'

In [4]:
class SqlString(SqlValue):
    def __init__(self, value):
        super().__init__(value)
        self.sql_type = 'str'

In [5]:
class SqlTimestamp(SqlValue):
    def __init__(self, value):
        super().__init__(value)
        self.sql_type = 'timestamp'

In [6]:
class SqlFloat(SqlValue):
    def __init__(self, value):
        super().__init__(value)
        self.sql_type = 'float'

In [7]:
class SqlJSONObj(SqlValue):
    def __init__(self, value):
        super().__init__(value)
        self.sql_type = 'JSONobj'

In [8]:
class SqlJSONArray(SqlValue):
    def __init__(self, value):
        super().__init__(value)
        self.sql_type = 'JSONarray'

In [9]:
class SqlJSONNumber(SqlValue):
    def __init__(self, value):
        super().__init__(value)
        self.sql_type = 'JSONnumber'

In [10]:
class SqlJSONString(SqlValue):
    def __init__(self, value):
        super().__init__(value)
        self.sql_type = 'JSONstring'

In [11]:
class SqlJSONbool(SqlValue):
    def __init__(self, value):
        super().__init__(value)
        self.sql_type = 'JSONbool'

In [12]:
class SqlExpression():
    def __init__(self, types, return_type, *operands):
        self.operands = list(operands)
        if len(types) != len(operands):
            print('err: Types array must be the same length as the number of operands')
            return
        
        
        for idx, operand in enumerate(self.operands):
            if not isinstance(operand, self.types[idx]):
                # Raise error
                
            if isinstance(operand, SqlExpression):
                if operand.return_type != self.types[idx]:
                    print(f"err: the opperand '{operand.to_sql()}' must be of return type {self.types[idx]} it is of return type {operand.return_type}")
                    self.operands[idx] = InvalidSqlExpression(operand)
                
            else:
                if not isinstance(operand, self.types[idx]):
                    print(f"err: the opperand '{operand.to_sql()}' must be of type {self.types[idx]} it is of type {type(operand)}")
                    continue
        
        

In [13]:
class InvalidSqlExpression():
    def __init__(self, expression):
        self.expression = expression
        
    def to_sql(self):
        return f"([INVALID SQL: {self.expression.to_sql()})"

In [14]:
class SqlAdd(SqlExpression, SqlInt):
    
    def __init__(self, lhs, rhs):
        self.types = [SqlInt, SqlInt]
        self.return_type = SqlInt
        self.lhs = lhs
        self.rhs = rhs
        
        super().__init__(self.types, self.return_type, self.lhs, self.rhs)
        
        self.lhs = self.operands[0]
        self.rhs = self.operands[1]
                
    def to_sql(self):
        return f"({self.lhs.to_sql()} + {self.rhs.to_sql()})"

In [15]:
class SqlCheckGreaterThan(SqlExpression, SqlInt):
    def __init__(self, lhs, rhs):
        self.types = [SqlInt, SqlInt]
        self.return_type = SqlBool
        self.lhs = lhs
        self.rhs = rhs
        
        super().__init__(self.types, self.return_type, self.lhs, self.rhs)
        
        self.lhs = self.operands[0]
        self.rhs = self.operands[1]
        
    def to_sql(self):
        return f"({self.lhs.to_sql()} > {self.rhs.to_sql()})"

In [16]:
class SqlCheckEquality(SqlExpression, SqlInt):
    def __init__(self, lhs, rhs):
        self.types = []
        self.return_type = SqlBool
        self.lhs = lhs
        self.rhs = rhs
        
        if (isinstance(lhs, SqlExpression)):
            self.types.append(lhs.return_type)
        else:
            self.types.append(type(lhs))
            
        if (isinstance(rhs, SqlExpression)):
            self.types.append(rhs.return_type)
        else:
            self.types.append(type(rhs))
            
            
        if self.types[0] != self.types[1]:
            print("Err: Cannont compute equality of unlike types:")
            print(self.types)
            return
        
        
        super().__init__(self.types, self.return_type, self.lhs, self.rhs)
        
        self.lhs = self.operands[0]
        self.rhs = self.operands[1]
        
    def to_sql(self):
        return f"({self.lhs.to_sql()} == {self.rhs.to_sql()})"

In [17]:
x = SqlInt('x')
y = SqlInt('y')
z = SqlInt('z')

In [18]:
exp1 = (x + y + z)

In [19]:
exp1.to_sql()

'((x + y) + z)'

In [20]:
exp1.return_type

__main__.SqlInt

In [21]:
exp2 = (x + y > z)

In [22]:
exp2.to_sql()

'((x + y) > z)'

In [23]:
exp2.return_type

__main__.SqlBool

In [24]:
exp3 = ((x > y) > z)

err: the opperand '(x > y)' must be of return type <class '__main__.SqlInt'> it is of return type <class '__main__.SqlBool'>


In [35]:
isinstance(exp2, SqlInt)

True

In [25]:
exp3.to_sql()

'(([INVALID SQL: (x > y)) > z)'

In [26]:
exp3 = ((x > y) > SqlString('asdf'))

err: the opperand '(x > y)' must be of return type <class '__main__.SqlInt'> it is of return type <class '__main__.SqlBool'>
err: the opperand 'asdf' must be of type <class '__main__.SqlInt'> it is of type <class '__main__.SqlString'>


In [27]:
type(exp3)

__main__.SqlCheckGreaterThan

In [28]:
expA = exp3 == exp3

In [29]:
expA.to_sql()

'((([INVALID SQL: (x > y)) > asdf) == (([INVALID SQL: (x > y)) > asdf))'

In [30]:
(x == x).to_sql()

'(x == x)'

In [31]:
(x == SqlString('asdf')).to_sql()

Err: Cannont compute equality of unlike types:
[<class '__main__.SqlInt'>, <class '__main__.SqlString'>]


'(x == asdf)'